In [98]:
import pandas as pd
import oracledb

conn_str = 'myuser/mypass123@localhost:1521/xepdb1'
conn=cur=None
try:
    conn = oracledb.connect(conn_str)
    cur = conn.cursor()

    query1 = """ SELECT TABLE_NAME,COLUMN_NAME,DATA_TYPE,CHAR_LENGTH,DATA_PRECISION,DATA_SCALE,NULLABLE
                 FROM ALL_TAB_COLUMNS
                 WHERE TABLE_NAME = 'CUSTOMERS'"""
    df_tgt = pd.read_sql(query1,conn)

    df_src = pd.read_excel('SOURCE_MAPPING.xlsx')

    df_merged = pd.merge(left=df_src,right=df_tgt,on=['TABLE_NAME','COLUMN_NAME'],how='inner',suffixes=("_SRC","_TGT"))
    # Since TABLE_NAME,COLUMN_NAME used for merging they apper only one in df_merged 
    df_merged['NULLABLE_SRC'] = df_merged['NULLABLE_SRC'].replace({'YES':'Y','NO':'N'})

    for ind,row in df_merged.iterrows():
    # COMPARE COLUMN NAMES
        src_cols = set(df_src['COLUMN_NAME']) # can't use df_merged for COLUMN_NAME comparision, because there is only 1 column for COLUMN_NAME
        tgt_cols = set(df_tgt['COLUMN_NAME'])
        if(src_cols != tgt_cols):
            print(f"Columns missing in TARGET: {src_cols-tgt_cols}")
            print(f"Columns extra in TARGET: {tgt_cols-src_cols}")

    # COMPARE DATA TYPES
        if(row['DATA_TYPE_SRC'] != row['DATA_TYPE_TGT']):
            print('❌ Failure.Mismatch bet datatypes!')
            print(f"{row['DATA_TYPE_SRC']}:{row['DATA_TYPE_TGT']}") 

    # COMPARE DATATYPE LENGTHS"
    # VARCHAR VALIDATION
        if(row['DATA_TYPE_SRC'] == 'VARCHAR2'):
            if(row['CHAR_LENGTH_SRC'] != row['CHAR_LENGTH_TGT']):
                print("❌ VARCHAR LENGTH MISMATCH")
                print(f"{row['COLUMN_NAME']}:{row['CHAR_LENGTH_SRC']}:{row['CHAR_LENGTH_TGT']}")

    # NUMBER VALIDATION
        if(row['DATA_TYPE_SRC'] == 'NUMBER'):
            if((row['DATA_PRECISION_SRC'] != row['DATA_PRECISION_TGT']) or (row['DATA_SCALE_SRC'] != row['DATA_SCALE_TGT'])):
                print("❌ NUMBER LENGTH MISMATCH")
                print(f"{row['COLUMN_NAME']}:{row['DATA_PRECISION_SRC']}:{row['DATA_PRECISION_TGT']}")  
                print(f"{row['COLUMN_NAME']}:{row['DATA_SCALE_SRC']}:{row['DATA_SCALE_TGT']}")  

    # DATE VALIDATION
        if(row['DATA_TYPE_SRC'] == 'DATE'):
            if(row['CHAR_LENGTH_SRC'] != row['CHAR_LENGTH_TGT']):
                print("❌ DATE LENGTH MISMATCH")
                print(f"{row['COLUMN_NAME']}:{row['CHAR_LENGTH_SRC']}:{row['CHAR_LENGTH_TGT']}")  

    # COMPARE NULLABLE
        if(row['NULLABLE_SRC'] != row['NULLABLE_TGT']):
            print("❌ Nullable mismatch.")
            print(f"{row['NULLABLE_SRC']}:{row['NULLABLE_TGT']}")

except Exception as err:
    print('Unable to connect to DB',err)    
finally:
    if(cur):
        cur.close()
    if(conn):
        conn.close()

C:\Users\HP\AppData\Local\Temp\ipykernel_7316\1818099220.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tgt = pd.read_sql(query1,conn)
